<img src="img/pacman.png"/>

# PACMAN (Search: BFS, IDS, A*)
## Introdudtion
Main goal of this project is to find solution for two pacman game with searching states space.<br> We read game map at first. Next, we define an initial states and by actions find next states deriving from this state. With checking this states, we find solution. <br> Our search algorithms are 1. BFS 2. IDS & 3. A*.
### Reading data
In below code section, __read_file_data__ open map file and read it line by line. It returns it's map in location 2D list.

In [2]:
def read_file_data(file_path):
    location = []
    with open(file_path) as file:
        line = file.readline()
        while line:
            location.append(line)
            line = file.readline()
    return location[:][:-1]
file_path = './testcases/test5'
location = read_file_data(file_path)

#### Helper functions
+ __find_initial_values__ function find initial values (P location(i,j), Q location(i,j), Food types and location(as a map)) for creating initial state. <br>
+ __distance_between__ function calculate manhatan distance between two points.

In [3]:
def find_initial_values(location):
    foods = {}
    i_p = 0
    j_p = 0
    i_q = 0
    j_q = 0
    for i in range(len(location)):
        for j in range(len(location[0])):
            if location[i][j] == '1' or location[i][j] == '2' or location[i][j] == '3':
                foods[(i,j)] = location[i][j]
            elif location[i][j] == 'P':
                i_p = i
                j_p = j
            elif location[i][j] == 'Q':
                i_q = i
                j_q = j
    return i_p, j_p, i_q, j_q, foods

def distance_between(i_1, j_1, i_2, j_2):
    return abs(i_2 - i_1) + abs(j_2 - j_1)

### Map problem to a search problem
Each state has 3 component:
    1. Location of P
    2. Location of Q
    3. Location of foods: 
       A dictionary that maps food locations(i,j) to it's type. (Key: tuple of i,j , Value: Food type)
#### Search components:
##### Initial state:
Get inital location of P and Q and foods from __get_initial_values__ function and create a node with this state. Set it's parent to _None_. It's cost is 0.
##### Goal state:
Goal state is state without any food. (Food dictionary got empty.)
##### Actions:
We should move P or Q in one of 4 dircetions.
    (1,2,3,4) move p (R,U,L,D)
    (5,6,7,8) move q (R,U,L,D)
Actions definitions can be found in moces array.
##### Transition model:
Moving P or Q change their location by adding _moves_ array to their previous location. If their new location, consists their compatible food type (1,3 for P and 2,3 for Q) they eat food and food removed from foods dictionary. 
If new location has incompatible food type or has wall, new state is invalid and new node can't created.
##### Path cost:
Each move costs 1. We should arrive goal in the least moves. <br> _(The optimal solution is the sequence of actions that gives the lowest path cost for reaching the goal)_

### State class:
#### Fields: 
- P location (i,j)
- Q location (i,j)
- Food dictionary
#### Methods:
- is_goal: Check foods dictionary is empty or not. 
- get_next_states: Generate state that derives from this state. Walls and poison block creating new states.
- h: Hueristic function estimates cost from this node to goal node.
- hash: This function calculates hash of this state for checking equality of state.
- show: This function gives a representation of state.

### Node class:
#### Fields:
- State of node
- Parent node of this node 
- Cost to reach this node
#### Methods:
- get_children: It gets next nodes of this node.
- bfs_hash: Returns hash of state.
- dfs_hash: Use for DFS. It adds cost to hash.
- h: Cost from this node to goal node.
- f: Path cost that includes this node.
- lt: Use for comparing nodes in heap.
- show: This function gives a representation of node.

In [4]:
moves = [(1,0), (0,1), (-1,0), (0,-1)]

class State:
    def __init__(self, i_p, j_p, i_q, j_q, foods):
        self.i_p = i_p
        self.j_p = j_p
        self.i_q = i_q
        self.j_q = j_q
        self.foods = foods
    
    def is_goal(self):
        return not self.foods
    
    def get_next_states(self, location):
        next_states = []
        for move in moves:
            # P Move
            next_i_p = self.i_p + move[0]
            next_j_p = self.j_p + move[1]
            if next_i_p < 0 or next_i_p >= len(location) or next_j_p < 0 or next_j_p >= len(location[0]):
                continue
            if location[next_i_p][next_j_p] == '%' or \
               ((next_i_p,next_j_p) in self.foods and self.foods[(next_i_p,next_j_p)] == '2') or \
               location[next_i_p][next_j_p] == 'Q':
                continue
            next_foods = self.foods.copy()
            if (next_i_p,next_j_p) in self.foods and \
                (self.foods[(next_i_p,next_j_p)] == '1' or (self.foods[(next_i_p,next_j_p)] == '3')):
                del next_foods[(next_i_p,next_j_p)]
            next_states.append(State(next_i_p, next_j_p, self.i_q, self.j_q, next_foods))
        for move in moves:
            # Q Move
            next_i_q = self.i_q + move[0]
            next_j_q = self.j_q + move[1]
            if next_i_q < 0 or next_i_q >= len(location) or next_j_q < 0 or next_j_q >= len(location[0]):
                continue
            if location[next_i_q][next_j_q] == '%' or \
               ((next_i_q,next_j_q) in self.foods and self.foods[(next_i_q,next_j_q)] == '1') or \
               location[next_i_p][next_j_p] == 'Q':
                continue
            next_foods = self.foods.copy()
            if (next_i_q,next_j_q) in self.foods and \
               (self.foods[(next_i_q,next_j_q)] == '2' or self.foods[(next_i_q,next_j_q)] == '3'):
                del next_foods[(next_i_q,next_j_q)]
            next_states.append(State(self.i_p, self.j_p, next_i_q, next_j_q, next_foods))
        return next_states
    
#     def h(self):
#         return len(self.foods)
    
#     def h(self):
#         distances = []
#         for food_loc, food_type in self.foods.items():
#             if food_type == '1' or food_type == '3':
#                 distances.append(distance_between(food_loc[0], food_loc[1], self.i_p, self.j_p))
#             if food_type == '2' or food_type == '3':
#                 distances.append(distance_between(food_loc[0], food_loc[1], self.i_q, self.j_q))
#         if not distances:
#             return 0
#         return min(distances)
            
    def h(self):
        distances = []
        for food_loc, food_type in self.foods.items():
            if food_type == '1':
                distances.append(distance_between(food_loc[0], food_loc[1], self.i_p, self.j_p))
            if food_type == '2':
                distances.append(distance_between(food_loc[0], food_loc[1], self.i_q, self.j_q))
            if food_type == '3':
                min_dist = distance_between(food_loc[0], food_loc[1], self.i_p, self.j_p)
                other_dist = distance_between(food_loc[0], food_loc[1], self.i_q, self.j_q)
                if other_dist < min_dist:
                    min_dist = other_dist
                distances.append(min_dist)
        if not distances:
            return 0
        return sum(distances)
    
    def __hash__(self):
        return hash((self.i_p, self.j_p, self.i_q, self.j_q, hash(frozenset(self.foods))))

    def show(self):
        print("P: ", self.i_p, self.j_p)
        print("Q: ", self.i_q, self.j_q)
        print("Foods: ", self.foods)
                
class Node:
    def __init__(self, state, parent=None, cost=0):
        self.state = state
        self.parent = parent
        self.cost = cost
        
    def get_children(self, location):
        return [Node(state, self, self.cost+1) for state in self.state.get_next_states(location)]
    
    def bfs_hash(self):
        return hash(self.state)
    
    def dfs_hash(self):
        return hash((self.state.i_p, self.state.j_p, self.state.i_q, self.state.j_q, 
                     hash(frozenset(self.state.foods)), self.cost))
    
    def h(self):
        return self.state.h()
    
    def f(self):
        return self.h() + self.cost
    
    def __lt__(self, other):
        if self.f() < other.f() or self.h() < other.h() or self.cost < other.cost:
            return True
        else:
            return False
    
    def show(self):
        self.state.show()
#         if self.parent:
#             print("parent: ")
#             self.parent.show()
        print("Cost: ", self.cost)
        print("************")

## BFS search:
It traverses nodes level by level. It implements by a FIFO list. We don't see nodes that viewed or in frontier list again.<br>
We check is_goal when we want to add node to frontier. It helps to find solution much faster. It finds optimal solution because we tracerse nodes level by level.

In [5]:
from time import time

def bfs(location):
#     start = time()
#     all_states = 1
#     diff_states = 1
    frontier = []
    frontier_hashes = set()
    explored_hashes = set()
    initial_state = State(*find_initial_values(location))
    initial_node = Node(initial_state)
    if initial_state.is_goal():
        return initial_node
    frontier.append(initial_node)
    frontier_hashes.add(initial_node.bfs_hash())
    while True:
        if not frontier:
#             print(time()-start)
            return None # Failure
        node = frontier.pop(0)
        explored_hashes.add(node.bfs_hash())
        for child in node.get_children(location):
#             all_states += 1
            if child.bfs_hash() in frontier_hashes or child.bfs_hash() in explored_hashes:
                continue
#             diff_states += 1
            if child.state.is_goal():
#                 print(time()-start)
#                 print("All states: ", all_states)
#                 print("Diffrent states: ", diff_states)
                return child # Can return path to root
            frontier.append(child)
            frontier_hashes.add(child.bfs_hash())

solution = bfs(location)
if solution:
    solution.show()
else:
    print("Solution does not exists!")

P:  6 1
Q:  5 4
Foods:  {}
Cost:  13
************


### IDS Search:
It traverses nodes in depth. DFS doesn't calculate optimal path but in IDS, we use DFS subroutine in different depths. If depth of node (cost) reach desired depth, We don't add it to frontier list. In IDS we should check is_goal when we expand a node. If we don't find solution in a depth(None result), we increase depth. It checks visited and frontier nodes and don't add existed nodes. For node equallity, it checks cost too. Because it may reach to one node on different costs and can produce less cost paths. <br>
__dfs_with__ function performs DFS subroutine. <br>
__ids__ function is ids core functionality. It iterates over diffrent depth and run dfs subroutine.

In [7]:
def dfs_with(depth):
    initial_state = State(*find_initial_values(location))
    initial_node = Node(initial_state)
    frontier = [initial_node]
    frontier_hashes = {hash(initial_state)}
    visited_hashes = set()
#     all_states = 1
#     diff_states = 1
    while True:
        if not frontier:
            return None
        node = frontier.pop()
        if node.state.is_goal():
            return node
        visited_hashes.add(node.dfs_hash())
        for child in node.get_children(location):
#             all_states += 1
            if child.dfs_hash() in frontier_hashes or child.dfs_hash() in visited_hashes or depth < child.cost:
                continue
#             diff_states += 1
            frontier.append(child)
            frontier_hashes.add(child.dfs_hash())

In [8]:
from time import time

def ids(location):
    start = time()
#     all_states = 0
#     diff_states = 0
    depth = 0
    while True:
        result = dfs_with(depth)
#         all_states += all_s
#         diff_states += diff_s
        if result:
#             print(time()-start)
#             print("All states: ", all_states)
#             print("Diffrent states: ", diff_states)
            return result
        depth += 1
#         print(depth)
#     print(time()-start)
#     print("All states: ", all_states)
#     print("Diffrent states: ", diff_states)
    return None

solution = ids(location)
if solution:
    solution.show()
else:
    print("Solution does not exists!")

P:  6 3
Q:  5 4
Foods:  {}
Cost:  13
************


## A* Search:
In this algorithm, we choose the most promising node. (The node that consider to be best choice for rest of the path to goal. We estimate that final path includes this node.)
For implementing this algorihtm, we have frontier nodes heap to pickup the least cost node. I choose heap because we need to get minimum value in each step and heap is the most fit data structure with this need. Each get minimum values costs O(logn) in time.
In each iteration, We pop the least cost node and after checking that it's goal, we add it's children to our heap. We don't check frontier nodes because we need to update heap values and it costs. We add the node again with less cost and because it's search cost in order logn it's more efficient for us.
#### Least cost node: 
- We have cost for each node. The cost that we consumed to get to this node. (g(n))
- h(n) (Hueristic function) estimates cost from this node to goal node. It's admissible if estimated value for each node is less than it's real cost. I suggest three hueristics:
    1. Number of remaining foods : It's admissible because we need one action for each food at least. It calculated in O(1).
    2. Manhatan distance of the closest food to agents: We find closest food to agents. The more close food is, Agent reach to it's solution faster. It's admissible because agents should eat many foods and by eating one food we don't reach to our goal. It calculated in O(# of foods).
    3. Sum of manhatan distance of all foods to agents: Agents should eat all foods. This measure can't be proven to be admissible but it prones state space very much and our algorithm comes to it's solution very fast. In many example because of walls and poisons it gets admissible. It calculated in O(# of foods).
- This hueristics are almost admissible so they're optimal.

In [9]:
from time import time
from heapq import heappush, heappop

def a_star(location):
#     start = time()
#     all_states = 1
#     diff_states = 1
    frontier = []
    visited_hashes = set()
    initial_state = State(*find_initial_values(location))
    initial_node = Node(initial_state)
    heappush(frontier, (initial_node.f(), initial_node))
    while True:
        if not frontier:
#             end = time()
#             print(end - start)
#             print("All states: ", all_states)
#             print("Diffrent states: ", diff_states)
            return None
        node = heappop(frontier)[1]
        if node.state.is_goal():
#             end = time()
#             print(end - start)
#             print("All states: ", all_states)
#             print("Diffrent states: ", diff_states)
            return node
        visited_hashes.add(node.bfs_hash())
        for child in node.get_children(location):
#             all_states += 1
            if child.bfs_hash() in visited_hashes:
                continue
#             diff_states += 1
            heappush(frontier, (child.f(), child))

solution = a_star(location)
if solution:
    solution.show()
else:
    print("Solution does not exists!")

P:  6 1
Q:  5 4
Foods:  {}
Cost:  13
************


## Differences & Advantages
### Completness and optimality
All of this algorithms are complete and optimal. 
### Time complexity
BFS runs in O($ b^d $) where d is depth of solution. <br>
IDS runs in O($ b^d $) too but it's a bit slower than BFS because it runs DFS sunbroutines many times. <br>
A* runs in O(# of nodes that f(n) < reak min cost). It prunes search space much more than BFS and IDS with good hueristic function. So it maybe has less time cost than IDS and BFS. IDS and BFS doesn't prune state space and grow in all directions.
### Space Complexity
BFS gets O($ b^d $) space because it saves all the visited and frontier nodes till goal node. <br>
IDS gets O($ bd $) space because it just saves frontier nodes for one branch. IDS inherits good time complexity from bfs and good space complexity from dfs.<br>
A* get exponential space. (It gets O(# of nodes that f(n) < reak min cost) space.) <br>
So winner of this part is __IDS__.

## Benchmark Table
- For each test and algorithm, I run my code three times and get average from them. <br>
- You can see best moves number in goal depth column and our moves number in Diffrent visited states 

<img src="img/benchmark.png">

## Execution time vs Goal depth plot for each algorithm

<img src="img/BFS.png">

<img src="img/IDS.png">

<img src="img/AStar.png">